In [1]:
import numpy as np
import sys
import re
import unicodedata
import pandas as pd
import keras.utils as ku
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
file = open('customer_service_data.csv', encoding='utf-8').read()

In [3]:
words = pd.read_csv('customer_service_data.csv')

In [4]:
tweets = list(words.columns)

In [30]:
print(tweets)

['Woke up to a call from a data science masters program. Well... time to save up.', 'Interested in a job in #NewYork, NY? This could be a great fit. Click the link in our bio to apply: Data Science In… ', '. interested in opportunity to explore the Texas After Violence Project at scale, e.g., through top… ', 'TRASH CAN. THROWING WATER ON JIMMY FALLON CHEEK NOT FART FART NOT A GARDEN WIIU CLOWN DIARRHEA ROOT BEER. JUSTIN BI… ', 'DIARRHEA DIAPERS AND LOOK TYRA BANKS IS A DIARRHEA. BIG DATA SCIENCE TO EARLY AND OFTEN POO DUMB TURKEY DEALER. MEA… ', 'In a world fraught with corruption a programmer and big data ramble love  become(s) desire', "That's a good day to listen to Compton. Making Sense of Big Data's Big Impact.", 'US IN ACNE TURD JUST DIARRHEAED ON TURKEY POX PACIFIC RIM DID THE TACO INFLUENCER MARKETING THE STRUCTURE OF DATA S… ', 'DATA SCIENCE YOUR BEEHIVE AND GOSSIP ABOUT HIS ELECTRONICS BLOCK CHEESE PUPPY FECES DIARRHEA (USED INSTEAD AS DIARR… ', 'PL await further details on o

In [6]:
print(file)

Woke up to a call from a data science masters program. Well... time to save up.,"Interested in a job in #NewYork, NY? This could be a great fit. Click the link in our bio to apply: Data Science In… ",". interested in opportunity to explore the Texas After Violence Project at scale, e.g., through top… ",TRASH CAN. THROWING WATER ON JIMMY FALLON CHEEK NOT FART FART NOT A GARDEN WIIU CLOWN DIARRHEA ROOT BEER. JUSTIN BI… ,DIARRHEA DIAPERS AND LOOK TYRA BANKS IS A DIARRHEA. BIG DATA SCIENCE TO EARLY AND OFTEN POO DUMB TURKEY DEALER. MEA… ,In a world fraught with corruption a programmer and big data ramble love  become(s) desire,That's a good day to listen to Compton. Making Sense of Big Data's Big Impact.,US IN ACNE TURD JUST DIARRHEAED ON TURKEY POX PACIFIC RIM DID THE TACO INFLUENCER MARKETING THE STRUCTURE OF DATA S… ,DATA SCIENCE YOUR BEEHIVE AND GOSSIP ABOUT HIS ELECTRONICS BLOCK CHEESE PUPPY FECES DIARRHEA (USED INSTEAD AS DIARR… ,PL await further details on our forthcoming event : Em

In [7]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.rstrip().strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  #w = '<start> ' + w + ' <end>'
  return w

In [4]:
file = preprocess_sentence(file)

In [5]:
print(file)

thanks , hahaha that s amazing , anything he wanted , very lol , heck yes ! , let s party , hi there ! we re disappointed to hear this . dm us the restaurant address . also , please provide us with , we care , haven t heard an end date , perfect , large sized oof , some people never learn , sometimes . but it s tight getting paid to tweet about , yeah , those things were , imagine it tho , just doing our thing , let s do lunch , the main move is stepping up , soon , it s an old meme sir , but it checks out , thanks , they started it , soon ish . like months or so , yes , we already thoroughly dunked on it . , preach , come through , much better , thanks , aw , leave the baby alone , haven t been on in a minute , but we should play soon . need to knock the rust off . , makes sense tbh , prove it , better at picking places to eat , straight fax , same . it s sooooooo good , it just sorta happens to us , not gonna argue with you there , the rule is , don t want none , don t start none , w

In [5]:
t = Tokenizer()
t.fit_on_texts(tweets)

In [6]:
print(t)

In [7]:
# A dictionary of words and their counts.
print(t.word_counts)

# A dictionary of words and how many documents each appeared in.
print(t.word_docs)

# An integer count of the total number of documents that were used to fit the Tokenizer (i.e. total number of documents)
print(t.document_count)

# A dictionary of words and their uniquely assigned integers.
print(t.word_index)

OrderedDict([('ℹ️maintenance', 1), ('is', 958), ('complete', 38), ('and', 1674), ('the', 3292), ('na', 2), ('eu', 2), ('xboxone', 1), ('megaservers', 2), ('are', 588), ('available', 91), ('patch', 1), ('notes', 2), ('…', 279), ('we', 2001), ('aware', 33), ('of', 683), ('issues', 112), ('browsing', 1), ('by', 104), ('category', 3), ('in', 693), ('xbox', 30), ('one', 90), ('store', 254), ('our', 1204), ('teams', 25), ('looking', 46), ('into', 370), ('this', 1752), ('will', 373), ('upda…', 3), ("we've", 55), ('received', 108), ('word', 7), ('that', 988), ('maintenance', 20), ('for', 2329), ('efootball', 4), ('pes', 4), ('2020', 13), ('has', 175), ('completed', 4), ('thank', 686), ('you', 3723), ('all', 151), ('reports', 4), ('as…', 15), ('ℹ️the', 1), ('currently', 58), ('unavailable', 8), ('while', 42), ('perform', 1), ('eso', 1), ('begun', 2), ('servers', 2), ('hello', 609), ('to', 5043), ('set', 32), ('up', 136), ('a', 1662), ('callback', 15), ('from', 195), ('an', 220), ('customer', 62

In [8]:
print('Found %s unique tokens.' % len(t.word_index))

Found 6211 unique tokens.


In [9]:
def get_sequence_of_tokens(corpus):
    t.fit_on_texts(corpus)
    total_words = len(t.word_index) + 1
    
    input_sequences = []
    for line in corpus:
        token_list = t.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            
    return input_sequences, total_words
input_sequences, total_words = get_sequence_of_tokens(tweets)

In [10]:
input_sequences[:10]

[[3033, 20],
 [3033, 20, 313],
 [3033, 20, 313, 9],
 [3033, 20, 313, 9, 3],
 [3033, 20, 313, 9, 3, 2173],
 [3033, 20, 313, 9, 3, 2173, 9],
 [3033, 20, 313, 9, 3, 2173, 9, 2174],
 [3033, 20, 313, 9, 3, 2173, 9, 2174, 3034],
 [3033, 20, 313, 9, 3, 2173, 9, 2174, 3034, 2175],
 [3033, 20, 313, 9, 3, 2173, 9, 2174, 3034, 2175, 33]]

In [11]:
# pad sequences 
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes = total_words)
    
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(input_sequences)

In [12]:
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len - 1))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [31]:
#filepath = "model_weights_saved.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
#desired_callbacks = [checkpoint]

In [14]:
model.fit(predictors, label, epochs=25)

C:\Users\Andy\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
101768/101768 [==============================] - 522s 5ms/step - loss: 6.1074
Epoch 2/25
101768/101768 [==============================] - 540s 5ms/step - loss: 5.3779
Epoch 3/25
101768/101768 [==============================] - 554s 5ms/step - loss: 4.9373
Epoch 4/25
101768/101768 [==============================] - 585s 6ms/step - loss: 4.6331
Epoch 5/25
101768/101768 [==============================] - 602s 6ms/step - loss: 4.3801
Epoch 6/25
101768/101768 [==============================] - 623s 6ms/step - loss: 4.1929
Epoch 7/25
101768/101768 [==============================] - 618s 6ms/step - loss: 4.0437
Epoch 8/25
101768/101768 [==============================] - 615s 6ms/step - loss: 3.9216
Epoch 9/25
101768/101768 [==============================] - 631s 6ms/step - loss: 3.8167
Epoch 10/25
101768/101768 [==============================] - 628s 6ms/step - loss: 3.7264
Epoch 11/25
101768/101768 [==============================] - 614s 6ms/step - loss: 3.6456
Epoch 12/25
101768/

In [15]:
filename = "word_vec_model_weights_saved.hdf5"
model.save_weights(filename)
print("saved model weights")

saved model weights


In [16]:
def generate_text(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        token_list = t.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ''
        
        for word,index in t.word_index.items():
            if index == predicted:
                output_word = word
                break
                
        seed_text = seed_text + " " + output_word
        
    return seed_text.title()

In [21]:
print(generate_text("why can't i see this page", 120, model, max_sequence_len))

Why Can'T I See This Page We Can Help With Your Order Please Dm Us With Your Name And Address And 1 Conta… … 1 3 3 3 3 3 3 3 3 Confirming To Be Availab… I… I… I… Came To The Refer Confirming Confirming Have A Moment Confirming Be Able… At… Is The Top 2Nd Day… I… Is The Stream 1 2 20 Tue I… Is The Top Confirming Plea… The Store And Manned Ch… Tue Is A Branded Gt Months Is A P… 1 Affecting To The Store Management I'Ve… Is Been 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 A Scam Second The Enforcement Team… Is A Scam We Want
